In [3]:

import pandas as pd
import torch
from torch_geometric.data import Data


In [10]:
network_data = pd.read_csv('/home/student/FinalProject/PaperFeedback/Datasets/citation_network_data.csv')

In [14]:
network_data.columns

Index(['id', 'title', 'year', 'author_name', 'author_org', 'author_id',
       'n_citation', 'doc_type', 'reference_count', 'references', 'venue_id',
       'venue_name', 'venue_type', 'doi', 'keyword', 'volume', 'issue',
       'publisher', 'weight', 'indexed_keyword', 'inverted_index'],
      dtype='object')

In [41]:
def max_weight_keyword(keywords, weights):
    keys = str(keywords).split(';')
    weight_vector = [float(x) for x in str(weights).split(';')]
    return keys[weight_vector.index(max(weight_vector))]

network_data['primary_keyword'] = network_data[['keyword', 'weight']].apply(lambda row: max_weight_keyword(row['keyword'], row['weight']), axis=1)

In [42]:
len(set(network_data['primary_keyword']))

67051

In [ ]:
def build_graph_dataset(data, relation_type='citations' ,labels='primary_keyword'):
    edges = []

    if relation_type == 'citation':
        for _, row in data.iterrow():
            paper_id = row['id']
            cited_papers = eval(row['citations'])
            for cited_paper in cited_papers:
                edges.append([paper_id, cited_paper])

    elif relation_type == 'co-citation': 
        pass

    elif relation_type == 'bibliographic_coupling':
        pass

    elif relation_type == 'author_collaboration':
        pass

    else:
        raise Exception('Illegal relation type')           


    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    X = torch.eye(len(data))
    node_labels = pd.Categorical(data[labels]).codes
    y = torch.tensor(node_labels, dtype=torch.long)
    graph_dataset = Data(x=X, edge_index=edge_index, y=y)
    return graph_dataset  

    